# 1. Setup

In [ ]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/spr_4711/code


In [ ]:
from mobilkit.umni import *
from project import *

# 2. Process data

## 2.1. Zones
Simplify geometry for faster loading and processing in the dashboard.

In [1]:
zones = gpd.read_parquet('../data/zones/zones2010.parquet').set_crs(CRS_DEG)
zones['scale'] = zones['scale'].map(D(COUNTY='County', TRACT='Tract', BG='Block Group'))
zones = zones[['scale', 'geoid', 'name', 'geometry']].disp()

NameError: name 'gpd' is not defined

In [81]:
zones.geoid.str[:5].to_frame().merge(zones[['geoid', 'name']], 'left', 'geoid')

,geoid,name
0,18083,Knox
1,18175,Washington
2,18081,Johnson
3,18075,Jay
4,18039,Elkhart
...,...,...
6412,18097,Marion
6413,18097,Marion
6414,18097,Marion
6415,18097,Marion


In [ ]:
zones.geometry = zones.geometry.simplify(0.005)
zones.to_file(U.mkfile('../data/dashboard/zones.shp.zip'), driver='ESRI Shapefile')

## 2.2. SES

In [51]:
ses = pd.read_csv('../data/acs/acs2019.csv').disp(0)

6,407 rows x 25 cols; Memory: 1.5 MiB


,scale,geoid,popu,pop_density,p_minor,p_poc,p_nocollege,m_income,p_pov,p_pov150,p_pov200,p_snap,p_unemploy,p_noinsur,p_disabled,p_lowenglish,p_snglparent,p_crowded,p_renter,p_mobilehome,p_noveh,m_hhperveh,p_nowfh,p_transit,p_walkbike
,<object>,<int64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>,<float64>


In [83]:
ses_cols = D(
    popu='Population',
    pop_density='Population Density',
    p_minor='Proportion Minors',
    p_poc='People of Color',
    p_nocollege='Low Education',
    m_income='Income Disadvantage',
    p_pov='Poverty Rate',
    p_pov150='1.5x Poverty Rate',
    p_pov200='2x Poverty Rate',
    p_snap='SNAP Recipients',
    p_unemploy='Unemployment Rate',
    p_noinsur='No Health Insurance',
    p_disabled='People with Disabilites',
    p_lowenglish='Limited English Speaking',
    p_snglparent='Single-Parent Households',
    p_crowded='High-Occupancy Households',
    p_renter='Rented Households',
    p_mobilehome='Mobile Homes',
    p_novehicle='No Vehicle Households',
    m_hhperveh='Vehicle Availability',
    p_nowfh='Commuters',
    p_transit='Transit Commuters',
    p_walkbike='Walk/Bike Commuters',
)

In [84]:
ses2 = (
    ses.astype({'geoid': str})
    .melt(['scale', 'geoid'], var_name='indicator', value_name='value')
    .assign(indicator=lambda df: df.indicator.map(ses_cols))
    .assign(percentile=lambda df: 100 * (1 - df.groupby(['scale', 'indicator'])
                            ['value'].rank(pct=True).rename('rank')))
    .rename(columns=D(value=False, percentile=True))
    .drop(columns='scale').melt(['geoid', 'indicator'], var_name='percentile')
    .astype({x: CAT for x in ['geoid', 'indicator', 'percentile']})
).disp()

294,722 rows x 4 cols; Memory: 4.0 MiB


,geoid,indicator,percentile,value
,<category>,<category>,<category>,<float64>
0,18069,Population,False,36359.0


In [85]:
# ses2.to_csv('../data/dashboard/ses.csv', index=False)
ses2.to_parquet('../data/dashboard/ses.parquet')

## 2.3. EJ

In [82]:
ejs = pd.read_parquet('../data/ejs/ejs.parquet').disp()

356,286 rows x 7 cols; Memory: 5.8 MiB


,scale,is_ses,is_derived,is_pctile,variable,geoid,value
,<category>,<bool>,<bool>,<bool>,<category>,<category>,<float64>
0,TRACT,True,False,False,Total: Population,18001030100,4826.0
